# Tutorial 1: QPFAS: Single Experiment

In this tutorial we show how to run a single VQE calculation using the `qpfas` package.

Note this tutorial (and the others) can be converted into a python file with

`jupyter nbconvert --to script <name_of_notebook.ipynb>`

or a pdf with

`jupyter nbconvert --to pdf <name_of_notebook.ipynb>`

In [ ]:
import qpfas  # we begin by importing the package

## Load molecule & select basis
There are many default molecules available in the `qpfas/chemistry/default_molecules` directory

In [ ]:
print(f"Default Molecules are: \n{qpfas.chemistry.default_molecules()}")

Next we create a Molecule class, this contains all the information about the chemistry of the molecule (i.e. sufficient information to perform calculations).

Thus along with the molecule geometry we need to specify a basis and an active space.

The options for the basis are:
- `sto-3g`
- `3-21G` 
- `6-31g` 
- `cc-pVDZ`

In the workflow generally sto-3g should be used, as the other choices require a larger number of qubits.

Active Space Approximations restrict the set of orbitals and thus reduce the number of qubits. 
Options are: 
- `full`, the full active space
- `frozen_core`, assume the inner orbitals are occupied
- `noons_<tol>` (where `<tol>` is a small number e.g. `noons_0.01`), where we use the post-HF method MP2 to indicate which orbitals are the most necessary to represent the molecule. The smaller the tolerance the more orbitals included.
- A list of the active orbitals e.g. [0, 1, 2, ...]

Let's see how many qubits each molecule requires to simulate using the `sto-3g` basis and a full active space.

In [ ]:
for mol in qpfas.chemistry.default_molecules():
    n_spatial_orbitals = len(qpfas.workflow.create_molecule(mol, "sto-3g", "full").active_orbitals)
    n_qubits = n_spatial_orbitals * 2
    print(mol, n_qubits)

We'll create a H2 molecule, with the sto-3g basis and full active space.

In [ ]:
molecule_qpfas = qpfas.workflow.create_molecule("H2", "sto-3g", "full")

# below is how to implement the noons method with tolerance 0.01
#molecule_qpfas = qpfas.workflow.create_molecule("H2", "sto-3g", "noons_0.01")


print(molecule_qpfas)

## Get Tequila molecule & get the qubit hamiltonian
Next we create the Tequila molecule and generate the qubit hamiltonian using the Jordan-Wigner transformation.

In [ ]:
molecule_tq = qpfas.workflow.get_molecule_tq(molecule_qpfas, "jordan_wigner")
qubit_hamiltonian_tq = qpfas.workflow.molecule_make_hamiltonian(molecule_tq)

print(f"Number of electrons: {molecule_tq.n_electrons}\n"
      f"Number of qubits: {qubit_hamiltonian_tq.n_qubits}\n"
      f"Number of Hamiltonian terms: {len(qubit_hamiltonian_tq)}")

## Get ansatz
Next we create the ansatz, possible options are:
- hardware
- hardwareconserving
- uccsd
- kupccgsd

We also have to choose the depth of the ansatz.

In [ ]:
ansatz_method = "uccsd"
ansatz = qpfas.workflow.get_ansatz(molecule_tq, ansatz_method=ansatz_method, ansatz_depth=1)

Given an ansatz it's useful to compute a range of statistics about the circuit. 

In [ ]:
gate_stats = qpfas.workflow.get_gate_dict(ansatz)

for i in gate_stats:
    print(f"{i}: {gate_stats[i]}")

## Run VQE
Now we run the VQE and print out the results.

For the choice of optimizer there are gradient free methods such as 
- `NELDER-MEAD`
- `COBYLA`
- `POWELl`

Second order methods
- `BFGS` (`L-BFGS-B` uses less memory)
- `CG`

As well as first order methods
- `sgd`
- `momentum`
- `adam`

In [ ]:
vqe_result = qpfas.workflow.run_vqe(qubit_hamiltonian_tq, ansatz, optimizer="bfgs", initialization=ansatz_method)

## Compare to benchmarks
We can compute the ground state energy using classical methods so as to see how accurate our VQE method is.

In [ ]:
benchmark_energies = qpfas.workflow.compute_benchmark_energies(molecule_tq, benchmarks=["hf", "ccsd", "fci"])
print(benchmark_energies)

In [ ]:
print("Error from FCI: (vqe method)", (vqe_result.energy-benchmark_energies["fci_energy"]))

## Tapering
`qpfas` also supports tapering (https://arxiv.org/abs/1701.08213) combined with the hardward ansatz. In the tapering method, several qubits are removed due to symmetries in the Hamiltonian. The hardware ansatz is necessary as tapering transforms the Hamiltonian into a new Hamiltonian with a different number of particles.

For H2, by using tapering we can reduce the circuit from 4 qubits to 1.

The details of which qubits were removed can be found below in the `tapering_dict`

In [ ]:
ansatz, taper_result, tapering_dict = qpfas.workflow.run_tapering_vqe(qubit_hamiltonian_tq, 
                                                                       depth=1, 
                                                                       optimizer="bfgs")

In [ ]:
tapering_dict

In [ ]:
print("Error from FCI: (tapering method)", (taper_result.energy-benchmark_energies["fci_energy"]))

In [ ]:
gate_stats = qpfas.workflow.get_gate_dict(ansatz)

for i in gate_stats:
    print(f"{i}: {gate_stats[i]}")

## Adapt

Finally, `qpfas` supports the Adapt ansatz, see https://www.nature.com/articles/s41467-019-10988-2

Here gates are interatively added to the Ansatz until convergence.

In [ ]:
adapt_result = qpfas.workflow.run_adapt_vqe(qubit_hamiltonian_tq, 0.1, molecule_tq, "bfgs", "gccsd")

In [ ]:
print("Error from FCI: (adapt method)", (adapt_result.energy-benchmark_energies["fci_energy"]))

In [ ]:
gate_stats = qpfas.workflow.get_gate_dict(adapt_result.U)


for i in gate_stats:
    print(f"{i}: {gate_stats[i]}")